In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import re
import time

In [2]:
#execute browser for navigation
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)


In [3]:
#Save News title and paragraph
url_nasa_news = "https://mars.nasa.gov/news/"
browser.visit(url_nasa_news)
time.sleep(10)
html=browser.html
soup = BeautifulSoup(html, 'html.parser')
news_title = soup.find_all("div", class_= "content_title")[1].text
news_p = soup.find("div", class_= "article_teaser_body").text
print(news_title)
print(news_p)

6 Things to Know About NASA's Ingenuity Mars Helicopter
The first helicopter attempting to fly on another planet is a marvel of engineering. Get up to speed with these key facts about its plans.


In [4]:
#JPL Mars Space Images - Featured Image
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

# find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

# Use the base url to create an absolute url
featured_image_url = f'https://www.jpl.nasa.gov{img_url_rel}'
featured_image_url


'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17440_hires.jpg'

In [5]:
# Mars Weather

twitter_mars_url= "https://twitter.com/marswxreport?lang=en"

browser.visit(twitter_mars_url)

time.sleep(5)

html =browser.html
weathersoup = BeautifulSoup(html,'html.parser')

try:
    mars_weather = weathersoup.find("p", "tweet-text").get_text()
    mars_weather
    
except AttributeError:
    
    pattern =re.compile('InSight sol')
    mars_weather = weathersoup.find('span', text=pattern).text
    mars_weather

mars_weather

'InSight sol 580 (2020-07-14) low -88.8ºC (-127.8ºF) high -8.4ºC (16.8ºF)\nwinds from the WNW at 5.9 m/s (13.3 mph) gusting to 15.4 m/s (34.4 mph)\npressure at 7.80 hPa'

In [6]:
# Mars Facts
url_mars_facts = "https://space-facts.com/mars/"
mars_facts = pd.read_html(url_mars_facts)
mars_facts_df = pd.DataFrame(mars_facts[0])
mars_facts_df = mars_facts_df.rename(columns={0:"Description", 1:"Value"})
mars_facts_df.set_index('Description', inplace=True)

#store html table
html_table = mars_facts_df.to_html()
html_table.replace("\n", "")
print(html_table)

#save df as html file
mars_facts_df.to_html("mars_facts.html")

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [7]:
# Mars Hemispheres
mars_hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
hemispheres_response = requests.get(mars_hemispheres_url)
soup = BeautifulSoup(hemispheres_response.text, 'html.parser')

#image data
image_data = soup.find_all("div", class_="item")

#hemisphere dictionary
hemisphere_image_urls = []
for image in image_data:
    image_title = image.find("h3").text
    image_url = image.find("a", class_="itemLink product-item")['href']
    full_image_url = f'https://astrogeology.usgs.gov{image_url}'
    
    # find image src
    image_response = requests.get(full_image_url)
    image_soup = BeautifulSoup(image_response.text, 'html.parser')
    image_src = image_soup.find('img', class_="wide-image")["src"]
    img_src_url = f'https://astrogeology.usgs.gov{image_src}'
    
    #add image title and source url to hemisphere dictionary
    hemisphere_image_urls.append({"title":image_title, "img_url": img_src_url })
  
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [8]:
 browser.quit()